# Introduction to Data Analysis using Pandas

This is part two of simple data analysis using pandas. It consists of more advanced topics

In [1]:
import pandas as pd
from helpers.analysis_preprocessor import preprocess # imports the preprocess from helpers/analysis_preprocessor.py 
df = pd.read_csv('data/twitter_data_chatgpt.csv.bz2')

#### Preprocessing.. again.
Because we reloaded the data, we need to do the same preprocessing again. A good practice is to define a preprocessing function that runs through all the steps so you can just call preprocess(df). You can also add additional steps later. We added this preprocess functions to helpers/analysis_preprocessor.py  and imported it in the previous block

In [ ]:
df = preprocess(df)
df

# Filtering Data
Filtering is the process of subsetting a dataset based on a logical condition.
It allows you to zoom in on relevant data: analyze them, visualize them, discard them or do certain manipulations on them much. You will be using it a lot. 

Filtering is done by a list of Boolean values that has the same size as the dataframe you will filter on like `[True, True, False....]`. It is called a (boolean) mask. You use the mask `df[mask]` to return a new Dataframe with just the rows for which the mask has a True value.

For instance, let's build a mask that has values all set to True

In [3]:
mask = [True for value in range(0, df.shape[0])] # this creates a list of 500,000 Trues

(Note that this code is inefficient and you should create this list using `mask = np.ones(df.shape[0], dtype=bool)` but that's not important for now)

Then use this on our dataframe. We get the same thing.

In [ ]:
df[mask]

Let's try a mask where only the first value is True.

In [ ]:
mask = [True] + [False for value in range(1, df.shape[0])] # this creates a list of 500,000 Trues
df[mask]

You of course won't create masks manually but by applying some logic to the data itself.

Let's filter the tweets that acquired at least 1 engagement (remember our newly created column engagement counts which is likes plus retweets?)
For that we need to create a mask that has True for all the rows where the engagement_count is greater than 0. (or more than or equal to 1)
This is simple:

In [ ]:
df['engagement_count'] > 0 # df['engagement_count'] >= 1 works too

Than we just put this mask into brackets:

In [ ]:
df[df['engagement_count'] > 0]

You can filter by multiple conditions. 
You have to wrap each condition in parentheses and put logical operators like & (AND), | (OR).

For instance, suppose we want tweets that have some engagements but they should have AT LEAST ONE retweet so:

In [ ]:
df[df['engagement_count'] >= 1 & df['retweet_count'] >= 1]

Wait, what the hell happened? I told you to wrap each condition in parenthesis remember?

When you omit parentheses, Pandas evaluates the expression like `df['engagement_count'] >= (1 & df['retweet_count']) >= 1` because comparison operators such as >= have higher precedence than bitwise operators like &. So, we use parentheses to explicitly control the order of evaluation—just like in math—ensuring that each condition is evaluated before they are combined.

In [ ]:
df[(df['engagement_count'] >= 1) & (df['retweet_count'] >= 1)]

You can use the query method for complex conditions:

In [ ]:
df.query("engagement_count >= 1 and retweet_count >= 1")

Note that you can always define variables which are masks or queries for readability and to use them later

In [ ]:
at_least_one_rt_and_engagement = (df['engagement_count'] >= 1) & (df['retweet_count'] >= 1)
df[at_least_one_rt_and_engagement]

You can use string methods to filter string columns in a DataFrame. To do this, we need to use the .str accessor (..to tell Pandas that we are not applying the string method to the whole column but each string value in the column individual) (Pandas are not the smartest that's why they face extinction!)

For instance, let's filter the tweets that start with '@', these are replies:

In [ ]:
df[df.text.startswith('@')]

This did not work, because we needed to use the .str accessor

In [ ]:
df[df.text.str.startswith('@')]

Tweets that contain a link:

In [ ]:
df[(df.text.str.contains('https://')) | (df.text.str.contains('http://'))]

Pandas knows how to do element-wise comparison so you do not need to use .str for string comparisons

Let's find the tweets authored by me! Omg I'm busted :o 

In [ ]:
df[df.username == 'tugrulcanelmas']

What if we want to collect tweets from multiple users and thus, have to do multiple comparisons? We can go like `(df.username == 'tugrulcanelmas') | (df.username == 'elonmusk') | (....)` but that is just too much code. 

It's better to create a list of usernames and then use `isin`.
Let's get tweets from SICSS Organizers: @tugrulcanelmas, @Walid_Magdy, 'Bjoernross, @TVGsociologist, @ZeerakTalat

In [ ]:
SICSS_list = ['tugrulcanelmas', 'Walid_Magdy', 'Bjoernross', 'TVGsociologist', 'ZeerakTalat']
df[df.username.isin(SICSS_list)]

Seems like only Tuğrulcan and Walid tweeted, Zee migrated to BlueSky, Tod forgot he had an account. That's life.

Note that you can use the `isin` for other dtypes like integers but you usually don't.

### Filter & Modify
You can create new columns by filtering on existing data.
For that, you need to use `.loc` to access columns and then assign new values.
The first time you do this on undefined, pandas will create a new column. However, all unassigned values will be set to null. Therefore **you need to assign a default value first**

Let's say we want to create a new column named "tweet_type" that indicates whether if the tweet is a __retweet__, a __reply__ or a regular, plain __tweet__

In [17]:
# Create a new column with default value
df['tweet_type'] = 'tweet'

# Now we can use .loc to assign new values
# We use the .str.startswith() method to check if the text starts with 'RT' for retweets, 
df.loc[df['text'].str.startswith('RT'), 'tweet_type'] = 'retweet'
# We use the .str.startswith() method to check if the text starts with '@' for replies
df.loc[df['text'].str.startswith('@'), 'tweet_type'] = 'reply'

Note that .loc accessor is to access rows. Remember the first part where we use ids to access the corresponding rows? (e.g., `df.loc[1641213003260633088]`)
In this example, what we did was to indicate the rows that satisfy a condition (e.g., `df.loc[df['text'].str.startswith('@')]`), instead of explicitly giving them a list of ids. 
We then selected a column and assign it a new value, for the rows that we accessed based on that condition.

Exercise: Remember Analysis Part 1. How could we achieve the same result using apply()?

In [ ]:
df['tweet_type_byapply'] = ...
# check
df['tweet_type'] == df['tweet_type_byapply']

# Sorting
Sorting can be done in two ways: sort index or sort values. 
Sorting by index is faster but it sorts the index itself. 
Sorting by values uses the values (i.e., the columns) in the dataframe to sort the data. It is slower.

In our case, the data was sorted according to date but in the reverse chronological order for some reason.. Let's fix this.
We can sort the data by the date column or the index, which was the id. 
Since ids are ascending by date, they provide the same result as sorting by date.

In [ ]:
df.sort_index(inplace = True)
df

To sort on a specific column we follow the template `sort_values(by = 'COLUMN TO SORT',)`. 

Note that both sort index and sort values sort in ascending order by default. But you can change it by setting ascending = False. 

Let's sort tweets according to the number of engagements:

In [ ]:
df.sort_values(by = 'engagement_count', inplace = True, ascending = False)
df

# Aggregation & Group By
In data analysis, aggregation refers to the process of summarizing or condensing data. Instead of analyzing individual rows, aggregation allows us to see patterns across groups — e.g., the average number of engagements each user receives or the total number of tweets per day. It helps reveal hidden insights in the data, such as emergent events indicated by sudden spikes in tweet volume.

Basic Aggregation Functions:
- count(): Count non-null values
- sum(): Sum of values
- mean(): Average of values
- min()/max(): Minimum/maximum values
- std(): Standard deviation
- size(): Number of rows in a **group** (**including null values**)

For instance, let's see the tweet with the highest number of engagements:

In [ ]:
df[df.engagement_count == df['engagement_count'].max()]

## Group By Operation: 
The groupby() function allows you to split data into groups and perform operations on each group.

`df.groupby('column_name')` gives you a DataFrameGroupBy object - a special intermediate object that represents the grouping operation. It is meaningless by itself, you need to perform an operation: aggregation, iteration or filtering.

Use the following template for aggregation:
`df.groupby('column_name').aggregation_function()`

For instance, the number of rows for each user (which means the number of tweets each user posted), and then sort the values to see who babbled the most

In [ ]:
df.groupby('username').size()

Add sort_values to see who babbled the most

In [ ]:
df.groupby('username').size().sort_values(ascending = False)

The total number of engagement each user 

In [ ]:
df.groupby('username')['engagement_count'].sum().sort_values(ascending = False)

You can have multiple aggregation functions using `agg()``
Let's compute the total and average number of engagements each user receives: 

In [ ]:
df.groupby('username').agg({'engagement_count': ['sum', 'mean']})

You can also aggregate by multiple columns. 

For instance, the total number of engagement each user receives in each day:

In [ ]:
df.groupby(['username', 'created_at'])['engagement_count'].sum()

Notice that 000Dave tweeted multiple dates, starting in 16/01/2023 and received different number of engagement counts each day. Good job, I guess?

You can also define and apply a custom function after groupby.

For instance, let's get the ratio of total likes to total retweets received by each user

In [ ]:
def like_retweet_ratio(x):
    return x['like_count'].sum() / (x['retweet_count'].sum() + 1) # smoothing the denominator to avoid division by zero

df.groupby('username').apply(like_retweet_ratio)

You can iterate for each group if you need to. You usually better off if you use aggregation functions. However, in some cases you may need to run complex logic individually on each group's DataFrame. Or you may need to iterate for visualization purposes

Let's group by the tweet type and describe each group:

In [ ]:
for tweet_type, group in df.groupby('tweet_type'):
    print(f"Tweet type: {tweet_type}")
    group_size = group.shape[0]
    max_engagement = group['engagement_count'].max()
    mean_engagement = group['engagement_count'].mean()
    print(f"Group size: {group_size}")
    print(f"Max engagement: {max_engagement}")
    print(f"Mean engagement: {mean_engagement}")
    print("\n")

The mean engagement for regular tweets is the highest because there are many AI Influencers out there farming engagements with crappy tutorials. Pity. Anyway.

Lastly, you can filter group based on a condition.
For instance, keep the tweets from the users who received more than 1000 engagements in total stays.

In [ ]:
df.groupby('username').filter(lambda x: x['engagement_count'].sum() > 1000)

Note that this grouped the DataFrame by username, filtered out the groups that don't meet the condition, and then merged the remaining groups back into a single DataFrame.

# Joining & Merging 
In Part 1, we learnt how to concatanate two dataframes. This was simple, just two dataframes stacked on top of each other. However, remember that we had problems when columns differed - we had to align them.

We concatanated vertically: we added the rows of one dataframe to another. What if we wanted to concatanate horizontally? That is, what if we wanted to add the **columns**  of a dataframe? Well, just taking two dataframes and sticking with pd.concat() still works. But in such a scenario, we often need to align the two dataframe so the data in the two dataframes will be consistent. Moreover, we need to multiply the rows in one of the dataframe to match the other.

Consider the case where we collect additional data about Twitter users like their popularity so each row in our new dataframe consists of a username and their popularity. How can we append this data to our existing data?

That's where `join and merge` operations come in. They unite the dataframes. They empower them. They make us stand!

Let's suppose we have a user dataframe `user_df` that holds additional data about users. We don't have such a dataframe so I'll create one using aggregation. The dataframe will consists of usernames and their popularity. The total engagement count the users receive will be a proxy for user popularity.

In [ ]:
# Create a dataframe with the total engagement count for each user
user_df = df.groupby('username')['engagement_count'].sum().sort_values(ascending = False)

# Create a new dataframe with the username and popularity
user_df = pd.DataFrame({'username': user_df.index, 'popularity': user_df.values})
user_df.head()

Now we want to merge this with our original dataframe to incorporate popularity data. We use the following template: `first_dataframe.merge(second_dataframe, on = 'column_to_merge')`

In [ ]:
df.merge(user_df, on = 'username') # note that we did not use inplace = True yet so it did not change the original dataframe

In many cases, the same data may have different column names across different dataframes. My recommendation is to rename columns so they match before performing a merge. However, if you prefer not to rename columns or want to keep both columns, you can specify different columns to merge on during the merge operation.

For example, suppose the username column in user_df is named screen_name instead of username (which is how the Twitter API actually names usernames, so you'll often encounter this when analyzing Twitter data).

In [32]:
user_df.rename(columns = {'username': 'screen_name'}, inplace = True)

When merging dataframes with different column names, we specify which columns to use from each dataframe. The first dataframe is called "left" and the second is called "right". We use the parameters "left_on" to specify the column from the left dataframe and "right_on" to specify the column from the right dataframe.

In [ ]:
df.merge(user_df, left_on = 'username', right_on = 'screen_name')
# Note that we did not use inplace = True yet so it did not change the original dataframe


Notice that username and screen_name has the same data

## Join
Join is the special case of merge where you merge on the index of the dataframes instead of columns. It's more efficient than merge

The basic syntax is: `dataframe1.join(dataframe2)`

Let's suppose we have additional data related to tweets, their popularity. Since I do not have such data, I will create a fake one by setting the engagement count as tweet popularity.

In [ ]:
fake_df = df[['engagement_count']]
fake_df.rename(columns={'engagement_count': 'popularity'}, inplace=True)
fake_df

Since both df's and fake_df's index is id, they can be joined together:

In [ ]:
df.join(fake_df)

What if we joined fake_df, df and then merged with user_df?    
(temporarily dropping some columns so you can see the output better)

In [ ]:
temporarily_drop_those_columns_for_readability = ['hashtags', 'tweet_type', 'engagement_count', 'like_count', 'retweet_count', 'created_at', 'text']
df.join(fake_df).merge(user_df, left_on = 'username', right_on = 'screen_name').drop(columns = temporarily_drop_those_columns_for_readability)

If you would look at the rightmost part of the merged dataframe, you will see `"popularity_x"` and `"popularity_y"` and they are different! One indicates the tweet popularity and the other indicates user popularity! 

Pandas, realizing your mistake, hastily renamed them by appending suffixes _x and _y. _x indicates left and _y indicates right. You can explicitly indicate suffixes while merging. But my recommendation is to properly rename them before!

In [ ]:
df.join(fake_df).merge(user_df, left_on = 'username', right_on = 'screen_name', suffixes = ['_tweet', '_user']).drop(columns=temporarily_drop_those_columns_for_readability)

Now we have more descriptive suffixes, but again:

**Properly rename your columns before join and merge operations!**

## Left, Right, Inner, Outer.. what the heck?
In many (annoying) cases, your data may not match up as expected. One dataframe might be missing information, while another might contain more data than anticipated. For instance, your original dataframe consists of tweets collected in 2023. You collect additional user data from Twitter data from X in 2025. You discover that some users have been suspended and you could not collect their data. Your user dataframe is incomplete.

You have several options:
1. Keep the original dataframe as it is, but leave the user data as null for users whose information couldn’t be retrieved.
2. Remove the tweets from the users with incomplete data, so you only keep the data with non-null values

The first approach is called an **outer** join/merge, while the second approach is called **inner** join/merge. The following image sums up very well:

![joins](./data/joins.jpg)

- An inner join returns only the rows where both dataframes have matching keys — essentially a set intersection.
- A left outer join keeps all rows from the left dataframe, even if there's no matching row in the right dataframe.
- A right outer join does the opposite, keeping all rows from the right dataframe.
- A full outer join includes all rows from both dataframes, with unmatched columns filled with nulls.


Let's practice with the hypothetical scenario we just explained: suppose the user_df is incomplete. I will make this by slicing the user_df.

In [ ]:
user_df_incomplete = user_df[0:len(user_df)//2] 
user_df_incomplete

You determine the join/merge type by the `"how"` argument. The options are `inner, left, right, outer`. The default value is `inner`

Inspect the output of the inner join and left join. What do you observe?

In [ ]:
# inner
df.merge(user_df_incomplete, how = 'inner', left_on = 'username', right_on = 'screen_name')

In [ ]:
# left outer
df.merge(user_df_incomplete, how = 'left', left_on = 'username', right_on = 'screen_name')

In [ ]:
# right outer 
df.merge(user_df_incomplete, how = 'right', left_on = 'username', right_on = 'screen_name')

In [ ]:
# full outer 
df.merge(user_df_incomplete, how = 'outer', left_on = 'username', right_on = 'screen_name')

# Reshaping Data

## Exploding Lists
We have extracted the hashtags in each tweet and put it to the `"hashtags"` column. Notice that it contains a list as a single tweet may contain multiple hashtags:

In [ ]:
df.hashtags

You can access the elements inside list columns using apply()

In [ ]:
df['hashtags'].apply(lambda x: x[0])

... and you get your first error because there are tweets **without any hashtag**, which means their hashtags column is an empty list (hence list index out of range error)

A work around is that returning None (or a dummy value like "NOHASHTAG") for empty lists

In [ ]:
df['hashtags'].apply(lambda x: x[0] if len(x) > 0 else None)

What you will do with this hashtag column is up to you. You can convert each hashtag into a new column (e.g., `hashtag1`, `hashtag2`, `hashtag3`...) or concatante into a single string. 

A very nice trick is "exploding" the list: flattening the list by keeping only a single hashtag in the column but duplicating the rows for the tweets with multiple hashtags. This is useful when you later group by or filter on some hashtags.

In [ ]:
# Explode the DataFrame on the hashtags column
exploded_df = df.explode('hashtags')

# Display the first few rows to verify the explosion
print("Exploded DataFrame:")
print(exploded_df[['username', 'hashtags']].head(10))

Notice that johnvianny used 3 hashtags in the tweet with id `1613755402608381952` so we have three records of the same tweet

We now can do a simple group by operation to show the number of times each user used a certain hashtag

In [ ]:
# Group by username and hashtags, then count occurrences
hashtag_usage = exploded_df.groupby(['username', 'hashtags']).size().reset_index(name='count')

# Sort by count in descending order to see most frequent hashtag usage
hashtag_usage = hashtag_usage.sort_values('count', ascending=False)

# Display the top 10 most frequent hashtag usages
print("Top 10 most frequent hashtag usages by user:")
print(hashtag_usage.head(10))


MidJourneyAI_ seems to have used the same set of 5 hashtags 1221. You may be thinking that the code is wrong (like me). Well, no, MidJourneyAI_ appends the same hashtags to every tweets that's why the results are like this :))

In [ ]:
df.loc[df.username=='MidJourneyAI_', 'hashtags']

## Pivoting
The hashtag_usage dataframe is currently structured like `(user1,hashtag1,value),(user1,hashtag2,value),(user2,hashtag1,value)...` 
You see it looks like a flat matrix.       
You can convert it to a matrix where the rows represent the users, columns represent the hashtags and each cell indicate the usage (count). This is called pivoting. This is useful for visualization and also when applying techniques like clustering and dimensionality reduction that you will see later.

In our example there are too many users and hashtags so I will pivot on only the first 50 rows of the hashtag usage dataframe

In [ ]:
pivotted = hashtag_usage.head(50).pivot(index='username', columns= 'hashtags', values='count')
pivotted

You can reverse pivotting by "melting". Do not forget to drop rows with null values.

In [ ]:
melted_df = pivotted.reset_index().melt(id_vars='username', var_name='hashtag', value_name='count')
melted_df = melted_df.dropna().sort_values(by = 'count', ascending=False)
melted_df.head(10)

# Saving Data
You can easily save your data to a csv by `.to_csv(filename)`, e.g., `df.to_csv('tweets.csv')`

However, I have some tips to avoid frustration later

- **Named index**: If you're using an index and it has a name, you're good to go. For example, if you’ve set the index to 'id' (i.e., `df.set_index('id')`), then the index will be saved properly, and no extra work is needed.
- **Unnamed index:** If you’ve assigned a custom index (e.g., `df.index = ...`) without setting a name, make sure to add one while saving: `df.to_csv(filename, index_label='your_index_name')`. Otherwise, the index column will just be labeled "index" in the CSV.
- **No index / Default Pandas index:** If you are NOT using an index, which means you are using the index automatically assigned by pandas, use `df.to_csv(filename, index = False)` to avoid pandas saving its auto-generated index. If you do not do this, you will add an extra column everytime you resave the same file.
- **Annoying Characters:** If you have string fields, I recommend you to **strip "\n"s and "\r"s** from them and then save. They often break formatting in CSV files and create more problems then they solve.
- **Complex Fields:** If you have fields containing lists, dictionaries, or other complex data, save in JSON format (.to_json) rather than CSV. CSV isn’t designed for nested structures and will make reloading or parsing those fields harder.
- ** Save as Records:** If saving as JSON, pandas defaults to a data structure in which the whole data is packed in a single JSON object. This is speed and memory efficient but it is a pain if you want to read the data line by line. Thus, save to json by setting `orient = records` so each row will correspond to a separate json object. If you do this, you need to read them as `pd.read_json(filename, lines = True) later.
- **Compress:** Compress your csvs and jsons so they will take less space, using `compression = 'bz2'`

and the most important tip:

**NEVER OVERWRITE THE ORIGINAL DATA**

In [7]:
df['text'] = df.text.apply(lambda x: x.replace('\n', ' ').replace('\r', ' '))
df.to_json('twitter_data_chatgpt_v2.json.bz2', orient='records', compression = 'bz2')